In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms
import h5py
import copy
import time
# print(torch.__version__)
file_name = "../data/CIFAR10.hdf5"
data = h5py.File(file_name, "r")
# get metadata for data: [n for n in data.keys()]
x_train = np.float32(data["X_train"][:]).reshape(-1, 3, 32, 32)
y_train = np.int32(np.array(data["Y_train"]))
x_test = np.float32(data["X_test"][:]).reshape(-1, 3, 32, 32)
y_test = np.int32(np.array(data["Y_test"]))
data.close()

In [4]:
model = CNN_torch()
resume = './checkpoint.pth.tar'
checkpoint = torch.load(resume, map_location="cpu")
start_epoch = checkpoint['epoch']
best_train_acc = checkpoint['best_train_acc']
train_loss = checkpoint['train_loss']
train_accuracy_epoch = checkpoint['train_accuracy_epoch']
model.load_state_dict(checkpoint['state_dict'])
model.eval() # load parameter
use_cuda = torch.cuda.is_available()
batch_size = 100
model.train()
test_accuracy = []
for i in range(0, len(y_test), batch_size):
    x_test_batch = torch.FloatTensor(x_test[i:i+batch_size, :])
    y_test_batch = torch.LongTensor(y_test[i:i+batch_size])
    if use_cuda:
        data, target = Variable(x_test_batch).cuda(), Variable(y_test_batch).cuda()
    else:
        data, target = Variable(x_test_batch), Variable(y_test_batch)
    output = model.forward(data)
    mc_output = output
    for i in range(49):
        output = model.forward(data)
        mc_output += output
    loss = F.nll_loss(mc_output / 50, target)
    prediction = output.data.max(1)[1]
    accuracy = (float(prediction.eq(target.data).sum()) / float(batch_size))
    test_accuracy.append(accuracy)
accuracy_test = np.mean(test_accuracy)
print(accuracy_test)

RuntimeError: Error(s) in loading state_dict for CNN_torch:
	Missing key(s) in state_dict: "conv1.weight", "conv1.bias", "batch_norm1.weight", "batch_norm1.bias", "batch_norm1.running_mean", "batch_norm1.running_var", "conv2.weight", "conv2.bias", "conv3.weight", "conv3.bias", "batch_norm2.weight", "batch_norm2.bias", "batch_norm2.running_mean", "batch_norm2.running_var", "conv4.weight", "conv4.bias", "conv5.weight", "conv5.bias", "batch_norm3.weight", "batch_norm3.bias", "batch_norm3.running_mean", "batch_norm3.running_var", "conv6.weight", "conv6.bias", "conv7.weight", "conv7.bias", "batch_norm4.weight", "batch_norm4.bias", "batch_norm4.running_mean", "batch_norm4.running_var", "conv8.weight", "conv8.bias", "batch_norm5.weight", "batch_norm5.bias", "batch_norm5.running_mean", "batch_norm5.running_var", "fc1.weight", "fc1.bias", "fc2.weight", "fc2.bias". 
	Unexpected key(s) in state_dict: "module.conv1.weight", "module.conv1.bias", "module.batch_norm1.weight", "module.batch_norm1.bias